In [1]:
from recordclass import make_arrayclass, litelist
from random import randint

In [2]:
import tracemalloc
class Tracer:
    def __enter__(self):
        if tracemalloc.is_tracing():
            raise ValueError('nesting tracemalloc is not allowed')
        self.allocated = None
        tracemalloc.start()
        return self
    def __exit__(self, exc_type, exc_value, exc_traceback):
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.allocated = current

In [5]:
with Tracer() as t0:
   d0 = {i:None for i in range(5_000_000)}
print("dict:", t0.allocated // 1_000_000, 'Mb')
del d0, t0

dict: 307 Mb


In [6]:
with Tracer() as t1:
    d1 = {}
    for i in range(N):
        key = (randint(0,N), randint(0,N), randint(0,N))
        val = [randint(0,N) for i in range(10)]
        d1[key] = val
print("regular:", t1.allocated // 1_000_000, 'Mb')
del d1, t1

regular: 3387 Mb


In [7]:
Triple = make_arrayclass("Triple", 3, hashable=True)

with Tracer() as t2:
    d2 = {}
    for i in range(N):
        key = Triple(randint(0,N), randint(0,N), randint(0,N))
        val = litelist([randint(0,N) for i in range(6)])
        d2[key] = val
print("recordclass:", t2.allocated // 1_000_000, 'Mb')
del d2, t2

recordclass: 2107 Mb


In [8]:
import sys
print(sys.getsizeof((1,2,3)))
print(sys.getsizeof(Triple(1,2,3)))
print(sys.getsizeof([1,2,3]))
print(sys.getsizeof(litelist([1,2,3])))


80
40
96
72
